In [27]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from urllib.parse import urlencode
from collections import Counter
import csv


In [28]:
# CSV file name
csvFile = "JobList.csv"

# Base URL
domain = "https://cv.ee/et/search?"

# Query parameters
params = {
    "limit": "1300",
    "offset": "0",
    "keywords[0]": "",
    "fuzzy": "true",
    "towns[0]": "312",
    "suitableForRefugees": "false",
    "isRemoteWork": "true",
    "isHourlySalary": "false",
    "isQuickApply": "false",
    "sorting": "LATEST",
}

# Constructing the URL
url = f"{domain}{urlencode(params)}"

# Making the request and parsing the HTML
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")

In [29]:
# Find all elements
job_listing_boxes = soup.find_all(class_="vacancies-list__item")

# Initialize results list
results = []

# Parse all the elements
for item in job_listing_boxes:
    parsed = {}
    if title := item.find(class_="vacancy-item__title"):
        parsed["title"] = title.text
    if item_url := item.a["href"]:
        parsed["url"] = urljoin(url, item_url.split("Name:")[-1])
    if company := item.find(class_="vacancy-item__body").a:
        parsed["company"] = company.text
    if location := item.find(class_="vacancy-item__locations"):
        parsed["location"] = location.text
    if salary := item.find(class_="vacancy-item__salary-label"):
        parsed["salary"] = salary.text
    else: parsed["salary"] = " "
    if expiry := item.find(class_="vacancy-item__expiry"):
        parsed["expiry"] = expiry.text.split("Tähtaeg: ")[-1].strip()
    #if posted_on := item.find("span", class_="color-white-mute", text=re.compile("posted:", re.I)):
    #    parsed["posted_on"] = posted_on.text.split("Posted:")[-1].strip()
    results.append(parsed)

In [30]:
# Write list to CSV file. 
myFile = open(csvFile, 'w')
writer = csv.writer(myFile)
writer.writerow(['title', 'url', 'company', "location", "salary", "expiry"])
for dictionary in results:
    writer.writerow(dictionary.values())
myFile.close()

In [33]:
# Read the CSV file and extract data for analysis
with open(csvFile, 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    company_field = [row['company'] for row in reader]

# Count the occurrences of each company
company_counts = Counter(company_field)

# Find the top 15 companies
top_companies = company_counts.most_common(15)

print("Top 15 Companies:")
for company, count in top_companies:
    print(f"{company}: {count} occurrences")

Top 15 Companies:
Töötukassa vahendatud pakkumised: 359 occurrences
Rimi Eesti Food AS: 22 occurrences
SEB Pank: 18 occurrences
Selver AS: 18 occurrences
Luminor: 17 occurrences
Swedbank AS: 13 occurrences
Tallink Grupp AS: 12 occurrences
Wise: 11 occurrences
Kaitseressursside Amet: 11 occurrences
Telia Eesti AS: 10 occurrences
Eesti Energia AS: 9 occurrences
Xfly: 9 occurrences
Ida-Tallinna Keskhaigla: 9 occurrences
Tallinna Tehnikaülikool: 9 occurrences
Wallester AS: 8 occurrences


In [32]:
# Print CSV file
myFile = open(csvFile, 'r')
print("The content of the csv file is:")
print(myFile.read())
myFile.close()

The content of the csv file is:
title,url,company,location,salary,expiry
Finnish-speaking Customer Support Representative,https://cv.ee/et/vacancy/1137561/suppline-sia/finnish-speaking-customer-support-representative,Suppline SIA,Tallinn,Alates  € 2150,21.02.2024
Ärikinnisvara müügiosakonna juht,https://cv.ee/et/vacancy/1126641/silikaat-grupp-as/arikinnisvara-muugiosakonna-juht,Silikaat Grupp AS,Tallinn, ,27.01.2024
Projekti-/Objektijuht,https://cv.ee/et/vacancy/1137555/elteks-ehitus-ou/projekti-objektijuht,ELTEKS EHITUS OÜ,Tallinn, ,20.02.2024
Katselabori spetsialist-analüütik,https://cv.ee/et/vacancy/1133986/evikon-mci-ou/katselabori-spetsialist-analuutik,Evikon MCI OÜ,Tartu /  Kaugtöö,€ 2000 – 3000,12.02.2024
Jurist,https://cv.ee/et/vacancy/1137543/talentor-estonia/jurist,Talentor Estonia,Tallinn, ,02.02.2024
Andmekeskuste insener,https://cv.ee/et/vacancy/1133982/riigi-infokommunikatsiooni-sihtasutus/andmekeskuste-insener,Riigi Infokommunikatsiooni Sihtasutus,Tallinn /  Kaugtöö, ,02